# Installation

In [1]:
import Pkg; Pkg.activate(dirname(@__DIR__))

  Activating new project at `c:\Users\pedroripper\Documents\Github`


# Stock Prices

In [2]:
using DataFrames
using Statistics

In [3]:
assets = [:IBM, :WMT, :SEHI]

df = DataFrames.DataFrame(
    [
         93.043    51.826    1.063
         84.585    52.823    0.938
        111.453    56.477    1.000
         99.525    49.805    0.938
         95.819    50.287    1.438
        114.708    51.521    1.700
        111.515    51.531    2.540
        113.211    48.664    2.390
        104.942    55.744    3.120
         99.827    47.916    2.980
         91.607    49.438    1.900
        107.937    51.336    1.750
        115.590    55.081    1.800
    ],
    assets,
)

Row,IBM,WMT,SEHI
,Float64,Float64,Float64
1,93.043,51.826,1.063
2,84.585,52.823,0.938
3,111.453,56.477,1.0
4,99.525,49.805,0.938
5,95.819,50.287,1.438
6,114.708,51.521,1.7
7,111.515,51.531,2.54
8,113.211,48.664,2.39
9,104.942,55.744,3.12


# Solving

In [4]:
using JuMP
using QUBO
using PySA

In [5]:
function solve(
    config!::Function,
    df::DataFrame,
    λ::Float64 = 10.;
    optimizer = PySA.Optimizer
)
    # Number of assets
    n = size(df, 2)

    # Relative monthly return
    r = diff(Matrix(df); dims = 1) ./ Matrix(df[1:end-1, :])

    # Expected monthly return value for each stock
    μ = vec(Statistics.mean(r; dims = 1))

    # Covariance matrix
    Σ = Statistics.cov(r)

    # Build model
    model = Model(() -> ToQUBO.Optimizer(optimizer))

    @variable(model, 0 <= x[1:n] <= 1)
    @objective(model, Max, μ'x - λ * x' * Σ * x)
    @constraint(model, sum(x) == 1)

    config!(model)

    optimize!(model)

    return value.(x)
end

function solve(df::DataFrame, λ::Float64 = 10.; optimizer = PySA.Optimizer)
    return solve(identity, df, λ; optimizer)
end

solve (generic function with 4 methods)

In [6]:
solve(df, 50.0) do model
    JuMP.set_silent(model)
    JuMP.set_attribute(model, "n_replicas", 5)
end

/mnt/d/gits/QUBO-notebooks/.CondaPkg/env/lib/python3.10/site-packages/pysa/simulation.py:155: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  energies[k] += sweep(update_spin, couplings, local_fields,


3-element Vector{Float64}:
 0.9032258064516129
 0.06451612903225806
 0.03225806451612903